In [1]:
# imports
import re
import sys
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from itertools import product
from functools import partial


# constants
MIN_T = 3
MAX_T = 28
min_t = pd.Timedelta(days=MIN_T)
max_t = pd.Timedelta(days=MAX_T)

In [252]:
# RAW DFS
islts = pd.read_excel('PLT_all_stool_isolates.xlsx')
islts.rename(columns = {'Stool ID':'StoolID', 'VRE +/-':'VRE'}, inplace = True)
islts = islts[islts.StudyID.notnull()] #lose 2 samples
islts = islts[~islts.ESBLisolate.apply(lambda v: type(v) is str and "SAME AS" in v)]#remove 4 duplciates




# clean mdro type vars
islts['ESBL'] = islts.ESBLisolate.replace([0, 'N', 'No', '-', 'n', 'N (NO MAC GROWTH)', 'N (ODD)'], 'No').\
    replace(['?', '35379- FALSE + DUE TO BAD PLATES', '35387 (FALSE + DUE TO BAD PLATES)',
            '35393 (FALSE + DUE TO BAD PLATES)'], np.nan).\
     apply(lambda v: np.nan if type(v) is str and 'BOTH FALSE + DUE TO BAD PLATES' in v else v)

islts['KPC'] = islts.KPCisolate.replace(['N', 'n', '-', 0, 'N (AFTER SUB CX)', 'N (NO MAC GROWTH)'], 'No').\
                 replace(['?', 'N (SMALL BLUE?)', '?N '], np.nan)

islts['VRE'] = islts.VRE.replace(['N', '-', 'n', '0'], 'No').replace(['+ (NEG on reculture 3/31)', 'NEG on reculture 4/12'], np.nan).\
    replace(['+', '+ (not pink)', '+ (mauve + blue)', '+ (pink + white)', '+ (mauve)', '+ (white)'], 'Yes')
    
islts = islts.drop(['ESBLisolate', 'KPCisolate'], axis = 1)

missing = [302615, 303205, 303955, 304362, 311175, 31117, 312645, 319365, 320275, 320315, 320515, 320525, 32073, 32077, 32088, 32089, 32090, 32096, 32098, 32101, 32106, 32107, 32108, 32119, 32138, 32141, 321481, 321482, 32152, 32173, 32177, 32184, 32196, 32197, 32198, 32221, 32235, 32238]
print(len(missing))
stlbk = pd.read_excel('StoolBookDeidentified20241023.xlsx')
stlbk.rename({'Stool_ID': 'StoolID'}, inplace=True, axis=1)
stlbk = stlbk.loc[stlbk.StoolID.isin(missing), :].drop(['ESBL_Growth', 'KPC_Growth', 'VRE_Growth'], axis=1)
mkstr = lambda x: f'{int(x)},' if pd.notna(x) else ''
stlbk['VRE'] = stlbk.apply(lambda r: 'No' if r[['VRE1', 'VRE2', 'VRE3']].isnull().all() else mkstr(r.VRE1)+mkstr(r.VRE2)+mkstr(r.VRE3), axis=1).apply(lambda x: x[:-1] if x.endswith(',') else x)
stlbk['ESBL'] = stlbk.apply(lambda r: 'No' if r[['ESBL1', 'ESBL2', 'ESBL3']].isnull().all() else mkstr(r.ESBL1)+mkstr(r.ESBL2)+mkstr(r.ESBL3), axis=1).apply(lambda x: x[:-1] if x.endswith(',') else x)
stlbk['KPC'] = stlbk.apply(lambda r: 'No' if r[['KPC1', 'KPC2', 'KPC3']].isnull().all() else mkstr(r.KPC1)+mkstr(r.KPC2)+mkstr(r.KPC3), axis=1).apply(lambda x: x[:-1] if x.endswith(',') else x)
stlbk.drop(['VRE1', 'VRE2', 'VRE3', 'ESBL1', 'ESBL2', 'ESBL3', 'KPC1', 'KPC2', 'KPC3'], axis=1, inplace=True)
stlbk['TxDate'] = stlbk.StudyID.apply(lambda x: islts.loc[islts.StudyID == int(x), 'TxDate'].iloc[0] if not islts.loc[islts.StudyID == int(x), 'TxDate'].empty else pd.NaT)
stlbk.rename({'Sample_Date':'SampleDate'}, axis=1, inplace=True)
print(islts.shape)
print(stlbk.shape)
islts = pd.concat([islts, stlbk])
print(islts.shape)

# ADD SEQUENCE INFO
r = pd.read_pickle('RawMetadata.df')
islts['StoolIDstr'] = islts.StoolID.astype(str)
islts['seq2020'] = islts.StoolIDstr.isin(r[r.index.get_level_values(1) == '2020'].index.get_level_values(0))
islts['seq2022'] = islts.StoolIDstr.isin(r[r.index.get_level_values(1) == '2022'].index.get_level_values(0))
islts['seq2023'] = islts.StoolIDstr.isin(r[r.index.get_level_values(1) == '2023'].index.get_level_values(0))
islts['seqALL'] = islts['seq2020'] | islts['seq2022'] | islts['seq2023']
islts = islts.merge(r.groupby(level=0).sum()[['PostHGFRC2']], left_on='StoolIDstr', right_index=True, how = 'left')
islts = islts.drop('StoolIDstr', axis = 1)
islts

dups = islts.groupby(['StudyID', 'SampleDate']).apply(lambda x: x[x.seqALL] if (len(x) > 1 and x.seqALL.any()) else None)
sngltns = islts.groupby(['StudyID', 'SampleDate']).apply(lambda x: x if len(x) == 1 else None)
islts = pd.concat([sngltns, dups])
# CHECK THAT WHAT WE'RE DROPPING IS THE NON_SEQ DUPLICATES
##PTNT 186 IS WEIRD W 31331 31332
#dupstlid = [31012, 30522, 30100, 30176, 30947, 30327, 30483, 30496, 30919, 31098, 
#            31332, 30973, 31277, 31820, 31501, 31899, 31828, 31884, 31808, 32033]
#islts = islts[~islts.StoolID.isin(dupstlid)]
#islts.shape
#PATH_TO_PE_EXT = '/manitou/pmg/projects/korem_lab/Projects/ACU_PLT/mmmbp2/tmp_path2/EXT/extended_contigs'
#islts['PATH'] = islts.StoolID.apply(lambda x: f'{PATH_TO_PE_EXT}/{x}.pe_ext.fasta')
#
#
#
## ABX
#abx  = pd.read_excel('PLT_Abx_Courses_Tal2.xlsx').iloc[:, :4]
#tx = islts[['TxDate', 'StudyID']].dropna().drop_duplicates()
#tx.index = tx.StudyID
#tx = tx.TxDate
#abx['course_start_date'] = abx.apply(lambda r: tx[r.StudyID] + pd.Timedelta(days=r.course_start_daysposttx), axis=1)
#abx['course_end_date'] = abx.apply(lambda r: tx[r.StudyID] + pd.Timedelta(days=r.course_end_daysposttx), axis=1)
#abx['course_duration'] = (abx.course_end_daysposttx - abx.course_start_daysposttx) + 1
#abx
#

38
(2056, 7)
(25, 7)
(2081, 7)


/tmp/ipykernel_145721/952736823.py:52: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dups = islts.groupby(['StudyID', 'SampleDate']).apply(lambda x: x[x.seqALL] if (len(x) > 1 and x.seqALL.any()) else None)
/tmp/ipykernel_145721/952736823.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sngltns = islts.groupby(['StudyID', 'SampleDate']).apply(lambda x: x if len(x) == 1 else None)


In [81]:
# ABX FUNCTIONS
def course_days_to_date(abx_df, date):
    if abx_df.empty:
        return np.nan
    days = abx_df.loc[abx_df.course_end_date < date, 'course_duration'].sum()
    days = pd.Timedelta(days=days)
    if any((abx_df.course_start_date <= date) & (date <= abx_df.course_end_date)):
        days += (date - abx_df.loc[(abx_df.course_start_date <= date) & (date <= abx_df.course_end_date), :].course_start_date).sum() + pd.Timedelta(days=1)
    return days.days

def days_since_last_exposure(abx_df, date):
    if abx_df.empty:
        return np.nan
    if any((abx_df.course_start_date <= date) & (date <= abx_df.course_end_date)):
        # sample taken during exposure
        return 0
    if any(abx_df.course_end_date < date):
        return (date - abx_df.loc[abx_df.course_end_date < date,'course_end_date']).min().days
    
def days_exposure_btwn(abx_df, sample, future):
    if abx_df.empty:
        return np.nan
    ovlp_s = abx_df.course_start_date.apply(lambda x: max(x, sample))
    ovlp_e = abx_df.course_end_date.apply(lambda x: min(x, future))
    ovlp = (ovlp_e - ovlp_s).apply(lambda x: 0 if x.days < 0 else x.days + 1)
    return ovlp.sum()
    

In [97]:
# METADATA AND OUTCOME FUNCTIONS

def get_vars(mdro_span, g):
    StudyID = g.name
    g = g.sort_values('SampleDate')
    g['Clear'] = (g[mdro_span] == 'No').all(axis=1)
    g['Positive'] = g[mdro_span].apply(lambda v: (v != 'No') & pd.notnull(v), axis=1).any(axis=1)
    g = g[(g[['Clear', 'Positive']] == True).any(axis=1)]
    gj = g.join(g.shift(-1), rsuffix='_nxt')
    gj['tx_to_sample_days'] = (gj.SampleDate - gj.TxDate).apply(lambda x: x.days)
    gj['sample_to_future_days'] = (gj.SampleDate_nxt - gj.SampleDate).apply(lambda x: x.days)
    gj['course_days_total'] = abx.loc[abx.StudyID == StudyID, 'course_duration'].sum()
    gj['course_days_to_sample'] = gj.apply(lambda r: course_days_to_date(abx.loc[abx.StudyID == StudyID,:], r.SampleDate), axis=1)
    gj['course_days_to_future'] = gj.apply(lambda r: course_days_to_date(abx.loc[abx.StudyID == StudyID, :], r.SampleDate_nxt), axis=1)
    gj['course_days_since_last_exposure'] = gj.apply(lambda r: days_since_last_exposure(abx.loc[abx.StudyID == StudyID, :], r.SampleDate), axis=1)
    gj['course_days_between'] = gj.apply(lambda r: days_exposure_btwn(abx.loc[abx.StudyID == StudyID, :], r.SampleDate, r.SampleDate_nxt), axis=1)
    #print(gj.to_string())
    return gj

def get_outcome(w, timetx, maxabx, clrlgc, seqstat, min_reads, g):
    g = g.sort_values('SampleDate')
    w_filt  = (w[0] <= g.sample_to_future_days) & (g.sample_to_future_days <= w[1])
    
    
    tx_filt =  (timetx[0] <= g.tx_to_sample_days) & (g.tx_to_sample_days <= timetx[1])
    # if you don't care allow nan
    if timetx == (-np.inf, np.inf):
        tx_filt |= g.tx_to_sample_days.isna()
    
    abx_filt = g.course_days_between <= maxabx
    # if you don't care, allow nan
    if maxabx == np.inf:
        abx_filt |= g.course_days_between.isna()
    
    mrd_filt = g.PostHGFRC2 >= min_reads
    pos_filt = g.Positive
    
    if seqstat == 'seqALL':
        seq_filt = g.seqALL
    elif seqstat == 'seq2023':
        seq_filt = g.seq2023
    elif seqstat == 'seq2022':
        seq_filt = g.seq2022
    elif seqstat == 'seq2020':
        seq_filt = g.seq2020
    elif seqstat is None:
        seq_filt = np.full((g.shape[0],), True)
    else:
        raise Exception
    
    filt = w_filt & abx_filt & tx_filt & pos_filt & seq_filt & mrd_filt
        
    #print('remaining:',  (w_filt & tx_filt & abx_filt & seq_filt).sum())
    if clrlgc == 'allclr':
        outcome = g.apply(lambda r: 'Clearance' if r['Clear_nxt'] == True else 'Persistence' if r['Positive_nxt'] == True else np.nan, axis = 1)
        outcome.loc[~filt] = np.nan
    elif clrlgc == 'typedclr':
        outcome = g.apply(lambda r: 'Persistence' if any(np.all(np.vstack([r[['VRE_nxt', 'KPC_nxt', 'ESBL_nxt']].apply(lambda x: x != 'No' and pd.notnull(x)), r[['VRE', 'KPC', 'ESBL']].apply(lambda x: x != 'No' and pd.notnull(x))]), axis=0)) else 'Clearence', axis = 1)
        outcome.loc[~filt] = np.nan
    else:
        raise Exception
    return outcome
    
def allclr(g):
    g = g.sort_values('SampleDate')
    g = g[(g[['Clear', 'Positive']] == True).any(axis=1)]
    gj = g.join(g.shift(-1), rsuffix='_nxt')
    gj = gj[((gj.SampleDate_nxt - gj.SampleDate) >= min_t) & ((gj.SampleDate_nxt - gj.SampleDate) <= max_t)]
    gj['Outcome'] = gj.apply(lambda r: 'Clearance' if r['Clear_nxt'] == True else 'Persistence' if r['Positive_nxt'] == True else np.nan, axis = 1)
    #print(gj.to_string())
    return gj

def spfcclr(g):
    g = g.sort_values('SampleDate')
    g = g[(g[['Clear', 'Positive']] == True).any(axis=1)]
    gj = g.join(g.shift(-1), rsuffix='_nxt')
    gj = gj[((gj.SampleDate_nxt - gj.SampleDate) >= min_t) & ((gj.SampleDate_nxt - gj.SampleDate) <= max_t)]
    gj['Outcome'] = gj.apply(lambda r: 'Persistence' if any(np.all(np.vstack([r[['VRE_nxt', 'KPC_nxt', 'ESBL_nxt']].apply(lambda x: x != 'No' and pd.notnull(x)), r[['VRE', 'KPC', 'ESBL']].apply(lambda x: x != 'No' and pd.notnull(x))]), axis=0)) else 'Clearence', axis = 1)
    return gj

#def get_outcome(g):
#    g = g.sort_values('SampleDate')
#    g = g[(g[['Clear', 'Positive']] == True).any(axis=1)]
#    gj = g.join(g.shift(-1), rsuffix='_nxt')
#    gj = gj[((gj.SampleDate_nxt - gj.SampleDate) >= min_t) & ((gj.SampleDate_nxt - gj.SampleDate) < max_t)]
#    gj['Outcome'] = gj.apply(lambda r: 'Clearance' if r['Clear_nxt'] == True else 'Persistence' if r['Positive_nxt'] == True else np.nan, axis = 1)
#    return gj

In [107]:
# GRID COMPUTE METADATA
MIN_READS=5*10**6
MIN_SAMPLES=30
MIN_LEAST_REPR_LABEL=10
# ALL MDRO
windows = [(3, 28)]#, (3, 21), (7, 21), (7, 28), (3, 14)]
time_from_tx = [(-np.inf, np.inf)]
abxbtwn = [np.inf]
clearance_logic = ['allclr']
seq_stat = ['seqALL']#, 'seq2022']#, None]
mdro_span = ['VRE', 'ESBL', 'KPC']
gv = partial(get_vars, mdro_span)
outcome = islts.groupby('StudyID').apply(gv, include_groups=False).reset_index().drop('level_1', axis=1)
# CHECK: Will get 614 in Tal version when selecting Positive samples but commenting out window filter
for w, timetx, maxabx, clrlgc, sqst in product(windows, time_from_tx, abxbtwn, clearance_logic, seq_stat):
    # CHECK: when adding the window filter (3, 28), and selecting seq_stat None, you get 323, matching Tal version
    time_txnm = f'{timetx[0]}-{timetx[1]}' if timetx != (-np.inf, np.inf) else ('any')
    outcome_t = f'Outcome_allmdro_{clrlgc}_tx{time_txnm}_abx{maxabx if maxabx != np.inf else "any"}_w{w[0]}-{w[1]}_{sqst}'
    outcome_fun = partial(get_outcome, w, timetx, maxabx, clrlgc, sqst, MIN_READS)
    outcome[outcome_t] = outcome.groupby('StudyID').apply(outcome_fun, include_groups=False).reset_index().drop(['StudyID', 'level_1'], axis=1)
    if outcome[outcome_t].notna().sum() < MIN_SAMPLES or outcome[outcome_t].dropna().value_counts().min() < MIN_LEAST_REPR_LABEL:
        print('SKIPPED: ', outcome_t, outcome[outcome_t].notna().sum(), 'samples')
        outcome.drop(columns=[outcome_t], inplace=True)
        continue
    print('Samples: ', outcome[outcome_t].dropna().shape[0])
    print('Patients: ', len(outcome[outcome[outcome_t].notna()].StudyID.value_counts()))
    print(outcome[outcome_t].dropna().value_counts())
outcome.to_csv('Outcome_allmdro.csv', index=None)
    
    
## VRE SPECIFIC
#windows = [(3, 28)]#, (3, 21), (7, 21), (7, 28), (3, 14)]
#time_from_tx = [(-np.inf, np.inf), (0, 28)]
#abxbtwn = [np.inf]
#clearance_logic = ['allclr']
#seq_stat = ['seq2023']#, 'seq2020', None]
#mdro_span = ['VRE']
#gv = partial(get_vars, mdro_span)
#outcome_vre = islts.groupby('StudyID').apply(gv, include_groups=False).reset_index().drop('level_1', axis=1)
#print('samples:', outcome_vre[outcome_vre.Positive].shape[0])
#for w, timetx, maxabx, clrlgc, sqst in product(windows, time_from_tx, abxbtwn, clearance_logic, seq_stat):
#    time_txnm = f'{timetx[0]}-{timetx[1]}' if timetx != (-np.inf, np.inf) else ('any')
#    outcome_t = f'Outcome_vremdro_{clrlgc}_tx{time_txnm}_abx{maxabx if maxabx != np.inf else "any"}_w{w[0]}-{w[1]}_{sqst}'
#    outcome_fun = partial(get_outcome, w, timetx, maxabx, clrlgc, sqst)
#    outcome_vre[outcome_t] = outcome_vre.groupby('StudyID').apply(outcome_fun, include_groups=False).reset_index().drop(['StudyID', 'level_1'], axis=1)
#    if outcome_vre[outcome_t].notna().sum() < MIN_SAMPLES:
#        print('SKIPPED: ', outcome_t, outcome_vre[outcome_t].notna().sum(), 'samples')
#        outcome_vre.drop(columns=[outcome_t], inplace=True)
#        continue
#    print(outcome_vre[outcome_t].dropna().shape)
#    print(outcome_vre[outcome_t].dropna().value_counts())
#outcome_vre.to_csv('Outcome_vremdro.csv', index=None)
#

/tmp/ipykernel_145721/2111453174.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  outcome = islts.groupby('StudyID').apply(gv, include_groups=False).reset_index().drop('level_1', axis=1)


Samples:  120
Patients:  93
Outcome_allmdro_allclr_txany_abxany_w3-28_seqALL
Persistence    83
Clearance      37
Name: count, dtype: int64


In [70]:

print(outcome_vre[outcome_vre['Outcome_vremdro_allclr_tx0-10_abx10_w5-12_seq2023'].notna()].iloc[:10,:].to_string())

     StudyID  StoolID SampleDate     TxDate    VRE  seq2020  seq2022  seq2023 ESBL KPC                                                                                                          PATH  Clear  Positive  StoolID_nxt SampleDate_nxt TxDate_nxt VRE_nxt seq2020_nxt seq2022_nxt seq2023_nxt ESBL_nxt KPC_nxt                                                                                                      PATH_nxt Clear_nxt Positive_nxt tx_to_sample_days sample_to_future_days  course_days_total  course_days_to_sample  course_days_to_future  course_days_since_last_exposure  course_days_between Outcome_vremdro_allclr_txany_abx10_w5-12_seq2023 Outcome_vremdro_allclr_tx0-10_abx10_w5-12_seq2023
50        19    30661 2015-06-17 2015-06-13  35327    False     True     True   No  No  /manitou/pmg/projects/korem_lab/Projects/ACU_PLT/mmmbp2/tmp_path2/EXT/extended_contigs/30661.pe_ext.fasta.gz  False      True      30675.0     2015-06-25 2015-06-13   35343       False       False       Fals

In [198]:


#missing_from_stlbk = [e for e in missing if not any(stlbk.StoolID.isin([e]))]
#print(len(missing_from_stlbk))
#missing_from_stlbk

38


,StoolID,Sample_Date,StudyID,VRE,ESBL,KPC,TxDate
1049,31117,2016-04-04,230.0,No,No,No,2016-03-31
1954,32073,2019-11-14,396.0,No,No,No,NaT
1958,32077,2019-11-21,405.0,No,35985,No,NaT
1967,32088,2020-01-14,405.0,35987,35986,No,NaT
1968,32089,2020-01-14,421.0,No,No,No,NaT
1969,32090,2020-01-14,414.0,No,No,No,NaT
1974,32096,2020-01-29,405.0,No,No,No,NaT
1976,32098,2020-02-03,403.0,36009,No,No,NaT
1979,32101,2020-02-10,264.0,No,No,No,NaT
1983,32106,2020-02-21,398.0,No,No,No,NaT


In [154]:
islts.loc[islts.StoolID == 30004]

,StoolID,SampleDate,StudyID,TxDate,VRE,seq2023,seq2020,seq2022,seqALL,PostHGFRC2,ESBL,KPC,PATH
1771,30004,2014-04-07,5,NaT,35888,True,False,False,True,26265429.0,NaN,35022,/manitou/pmg/projects/korem_lab/Projects/ACU_P...


In [7]:
######### TAL VERSION ##########
islts = pd.read_excel('PLT_all_stool_isolates.xlsx')

islts.rename(columns = {'Stool ID':'StoolID', 'VRE +/-':'VRE'}, inplace = True)

islts = islts[islts.StudyID.notnull()] #lose 2 samples
islts = islts[~islts.ESBLisolate.apply(lambda v: type(v) is str and "SAME AS" in v)]#remove 4 duplciates
islts.head()

islts['ESBL'] = islts.ESBLisolate.replace([0, 'N', 'No', '-', 'n', 'N (NO MAC GROWTH)', 'N (ODD)'], 'No').\
    replace(['?', '35379- FALSE + DUE TO BAD PLATES', '35387 (FALSE + DUE TO BAD PLATES)',
            '35393 (FALSE + DUE TO BAD PLATES)'], np.nan).\
     apply(lambda v: np.nan if type(v) is str and 'BOTH FALSE + DUE TO BAD PLATES' in v else v)

islts['KPC'] = islts.KPCisolate.replace(['N', 'n', '-', 0, 'N (AFTER SUB CX)', 'N (NO MAC GROWTH)'], 'No').\
                 replace(['?', 'N (SMALL BLUE?)', '?N '], np.nan)

islts['VRE'] = islts.VRE.replace(['N', '-', 'n', '0'], 'No').replace(['+ (NEG on reculture 3/31)', 'NEG on reculture 4/12'], np.nan).\
    replace(['+', '+ (not pink)', '+ (mauve + blue)', '+ (pink + white)', '+ (mauve)', '+ (white)'], 'Yes')


islts = islts.drop(['ESBLisolate', 'KPCisolate'], axis = 1)

#PTNT 186 IS WEIRD W 31331 31332
dupstlid = [31012, 30522, 30100, 30176, 30947, 30327, 30483, 30496, 30919, 31098, 
            31332, 30973, 31277, 31820, 31501, 31899, 31828, 31884, 31808, 32033]

print(islts.shape)
islts = islts[~islts.StoolID.isin(dupstlid)]
print(islts.shape)
islts['Clear'] = (islts[['VRE', 'ESBL', 'KPC']] == 'No').all(1)
islts['Positive'] = islts[['VRE', 'ESBL', 'KPC']].applymap(lambda v: v != 'No' and pd.notnull(v)).any(axis=1)


(2056, 7)
(2036, 7)


/tmp/ipykernel_145721/418072466.py:32: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  islts['Positive'] = islts[['VRE', 'ESBL', 'KPC']].applymap(lambda v: v != 'No' and pd.notnull(v)).any(axis=1)


In [12]:
def get_outcomeTK(g):
    g = g.sort_values('SampleDate')
    g = g[(g[['Clear', 'Positive']] == True).any(axis=1)]
    gj = g.join(g.shift(-1), rsuffix='_nxt')
    gj = gj[((gj.SampleDate_nxt - gj.SampleDate) >= min_t) & ((gj.SampleDate_nxt - gj.SampleDate) < max_t)]
    gj['Outcome'] = gj.apply(lambda r: 'Clearance' if r['Clear_nxt'] == True else 'Persistence' if r['Positive_nxt'] == True else np.nan, axis = 1)
    return gj

In [11]:
outcome = islts.groupby('StudyID').apply(get_outcomeTK, include_groups=False).reset_index().drop('level_1', axis = 1)
outcome = outcome[outcome.Positive]
outcome

# NOTE, to get 323 rows, the same as with grid version, set <= max_t in get_outcomeTK() not < max_t

,StudyID,StoolID,SampleDate,TxDate,VRE,ESBL,KPC,Clear,Positive,StoolID_nxt,SampleDate_nxt,TxDate_nxt,VRE_nxt,ESBL_nxt,KPC_nxt,Clear_nxt,Positive_nxt,Outcome
0,5,30004,2014-04-07,NaT,35888,NaN,35022,False,True,30011.0,2014-04-17,NaT,35892,NaN,No,False,True,Persistence
1,6,30017,2014-04-28,2014-03-26,35893,NaN,"35024, 35025",False,True,30019.0,2014-05-05,2014-03-26,35894,35027,35026,False,True,Persistence
2,6,30019,2014-05-05,2014-03-26,35894,35027,35026,False,True,30029.0,2014-05-20,2014-03-26,35834,No,No,False,True,Persistence
3,6,30029,2014-05-20,2014-03-26,35834,No,No,False,True,30036.0,2014-06-04,2014-03-26,35835,No,No,False,True,Persistence
6,9,30006,2014-04-11,2014-04-07,35889,NaN,No,False,True,30010.0,2014-04-16,2014-04-07,35805,NaN,No,False,True,Persistence
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,423,31920,2018-12-20,NaT,35954,No,No,False,True,31927.0,2019-01-03,NaT,No,P:35958 B:35959,35957,False,True,Persistence
787,423,31927,2019-01-03,NaT,No,P:35958 B:35959,35957,False,True,31937.0,2019-01-17,NaT,No,No,No,True,False,Clearance
788,424,31982,2019-03-29,NaT,35965,No,No,False,True,31986.0,2019-04-10,NaT,35966,No,No,False,True,Persistence
800,439,32019,2019-06-24,NaT,35974,35971,No,False,True,32024.0,2019-07-03,NaT,35976,35978,No,False,True,Persistence


In [144]:
oldX = pd.read_csv('mdro_pos.csv')
x_samples=[int(e) for e in """30661
30038
31068
30049
30371
30891
30104
30434
30537
30542
30266
31022
30512
30927
31140
31581
30939
30788
31141
31224
31307
31110
31194
31268
31777
30572
30096
30165
30094
30230
30355
30316
30688
30394
30543
30665
30582
30853
30758
30848
30725
30875
31227
31424
31708
31456
32015""".splitlines()]
oldX = oldX.loc[oldX['Stool ID'].isin(x_samples), :]
islts[islts.StoolID.isin(oldX['Stool ID'])].PATH.to_csv('oldX.list', header=None, index=None)


In [145]:
COPAN_INI_TMPLT = """# enter all config in 'key = value' format
app_name = cpg
log_file_dir = logs # path relative to the executable
log_level = 0 # possible values are 1ebug=0(includes Info and Error), Info=1(includes Error), Error=2, None=3
log_to = 0 # possible values are Console=0, File=1, ConsoleAndFile=2
sample_list = /burg/pmg/users/ic2465/Projects/MANU_copangraph/data/CAMISIMCoassembly/XNAME.txt
graph_name = XNAME # name of graph prefix
out_dir = /burg/pmg/users/ic2465/Projects/MANU_copangraph/data/CAMISIMCoassembly/
divergence_threshold = XSD
num_threads = 32
max_separation = 75
window_size = 10
kmer_size = 15
min_homology_overlap = 1000
min_contiguity_overlap = 60
max_jump = 200 # largest allowed gap within an alignment
high_freq_kmer_filter = 1e-5 # remove the top 1-x percentile kmers where x is input
fasta_file_ext = .fasta 
gfa_file_ext = .gfa
node_color_file_ext = .ncolor.csv
edge_color_file_ext = .ecolor.csv
extended_contigs = true
sensitive_mode = true"""

copan_05 = COPAN_INI_TMPLT.replace('XNAME', 'oldX').replace('XSD', '0.05')
copan_02 = COPAN_INI_TMPLT.replace('XNAME', 'oldX').replace('XSD', '0.02')
with open('oldX_05.ini', 'w') as f:
    f.write(copan_05)
with open('oldX_02.ini', 'w') as f:
    f.write(copan_02)
